In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from dotenv import load_dotenv
from openai import OpenAI

from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from utils import add_new_customer, check_existing_customer_email, validate_email, validate_phone, check_existing_customer_name, check_existing_customer_phone

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Get the List of Tools
tools = [add_new_customer, check_existing_customer_email, validate_email, check_existing_customer_name, validate_phone, check_existing_customer_phone]

# Initialize the agent with tools and an LLM
llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    You are an intelligent assistant with access to the following tools:
    1. AddCustomer: Add a new customer with name, email, and phone.
    2. CheckCustomerEmail: Check if a customer exists using their email.
    3. CheckCustomerName: Check if a customer exists using their name.
    3. CheckCustomerPhone: Check if a customer exists using their phone number.
    4. ValidateEmail: Validate the format of an email address.
    5. ValidatePhone: Validate the format of a phone number.

    For any request to add a customer:
    - First, verify if the customer already exists using their email or name or phone number
    - Only if the customer does not exist, proceed to use AddCustomer if customer name, email, and phone number is given.

    Respond to the following user query using the tools appropriately:
    {query}
    """
)

In [ ]:
# User query example
user_query = "What is your name"
formatted_prompt = prompt_template.format(query=user_query)

# Run the agent
response = agent.invoke(formatted_prompt)
print("Response:", response['output'])